<a href="https://colab.research.google.com/github/Jamie787/weatherwise-jaime/blob/main/weatherwise_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Intelligent Weather Analysis & Advisory System

**Author:** Jaime  
**Project:** Weather Advisor Application  
**Description:** A comprehensive weather application combining real-time weather data with natural language processing for intuitive user interactions.

---

## 📋 Table of Contents

1. [Setup and Configuration](#setup)
2. [Weather Data Functions](#weather-data)
3. [Visualization Functions](#visualization)
4. [Natural Language Processing](#nlp)
5. [User Interface](#ui)
6. [Main Application Logic](#main-app)
7. [Testing and Examples](#testing)

---

In [2]:
# 🌅 Test: manual cell (wind + sunrise/sunset for Perth & Lisboa)

import requests
from datetime import datetime

API_KEY = "demo"  # demo key for illustration only
cities = ["Perth", "Lisbon"]
url = "https://api.open-meteo.com/v1/forecast"

for city in cities:
    print(f"\n--- {city.upper()} ---")
    params = {
        "latitude": -31.95 if city == "Perth" else 38.72,
        "longitude": 115.86 if city == "Perth" else -9.14,
        "current_weather": True,
        "timezone": "auto",
        "daily": ["sunrise", "sunset"]
    }

    response = requests.get(url, params=params)
    data = response.json()
    weather = data.get("current_weather", {})
    daily = data.get("daily", {})

    print(f"Temperature: {weather.get('temperature', 'N/A')}°C")
    print(f"Wind: {weather.get('windspeed', 'N/A')} km/h")
    print(f"Wind direction: {weather.get('winddirection', 'N/A')}°")
    print(f"Sunrise: {daily.get('sunrise', ['N/A'])[0]}")
    print(f"Sunset: {daily.get('sunset', ['N/A'])[0]}")



--- PERTH ---
Temperature: 17.5°C
Wind: 18.8 km/h
Wind direction: 177°
Sunrise: 2025-10-26T05:24
Sunset: 2025-10-26T18:36

--- LISBON ---
Temperature: 19.3°C
Wind: 9.7 km/h
Wind direction: 45°
Sunrise: 2025-10-26T06:57
Sunset: 2025-10-26T17:43


## 📦 Setup and Configuration <a id='setup'></a>

Import required packages and configure the environment.

In [ ]:
# Install required packages (uncomment if running in Google Colab)
# !pip install fetch-my-weather hands-on-ai pyinputplus matplotlib requests

In [ ]:
# Import standard libraries
import os
import re
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any

# Import third-party libraries
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pyinputplus as pyip
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Configure hands-on-ai
os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'

# API key setup (if needed)
# Uncomment the following line to enter your API key interactively
# os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

print("✅ All packages imported successfully!")

## 🌤️ Weather Data Functions <a id='weather-data'></a>

Functions for retrieving and processing weather data from the fetch-my-weather API.

In [ ]:
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
              Returns None if an error occurs
    """
    try:
        # Validate forecast_days parameter
        forecast_days = max(1, min(5, forecast_days))

        # Fetch weather data using fetch-my-weather
        weather_response = get_weather(
            location=location,
            format="json",
            units="m"  # metric units
        )

        # Check if the response is an error string
        if isinstance(weather_response, str) and weather_response.startswith("Error:"):
            print(f"❌ {weather_response}")
            return None

        # Extract and structure the data
        weather_data = {
            'location': location,
            'current_condition': None,
            'forecast': [],
            'area_info': None
        }

        # Extract current condition
        if hasattr(weather_response, 'current_condition') and weather_response.current_condition:
            current = weather_response.current_condition[0]
            weather_data['current_condition'] = {
                'temp_C': current.temp_C,
                'temp_F': current.temp_F,
                'feels_like_C': current.FeelsLikeC,
                'feels_like_F': current.FeelsLikeF,
                'humidity': current.humidity,
                'weather_desc': current.weatherDesc[0].value if current.weatherDesc else "Unknown",
                'wind_speed_kmph': current.windspeedKmph,
                'wind_dir': current.winddir16Point,
                'precip_mm': current.precipMM,
                'visibility': current.visibility,
                'uv_index': current.uvIndex
            }

        # Extract area information
        if hasattr(weather_response, 'nearest_area') and weather_response.nearest_area:
            area = weather_response.nearest_area[0]
            weather_data['area_info'] = {
                'area_name': area.areaName[0].value if area.areaName else "Unknown",
                'country': area.country[0].value if area.country else "Unknown",
                'region': area.region[0].value if area.region else "Unknown"
            }

        # Extract forecast data (limit to requested days)
        if hasattr(weather_response, 'weather') and weather_response.weather:
            for day in weather_response.weather[:forecast_days]:
                forecast_day = {
                    'date': day.date,
                    'max_temp_C': day.maxtempC,
                    'min_temp_C': day.mintempC,
                    'max_temp_F': day.maxtempF,
                    'min_temp_F': day.mintempF,
                    'avg_temp_C': day.avgtempC,
                    'avg_temp_F': day.avgtempF,
                    'total_snow_cm': day.totalSnow_cm,
                    'sun_hour': day.sunHour,
                    'uv_index': day.uvIndex,
                    'hourly': []
                }

                # Extract hourly data
                for hour in day.hourly:
                    hourly_data = {
                        'time': hour.time,
                        'temp_C': hour.tempC,
                        'temp_F': hour.tempF,
                        'feels_like_C': hour.FeelsLikeC,
                        'feels_like_F': hour.FeelsLikeF,
                        'weather_desc': hour.weatherDesc[0].value if hour.weatherDesc else "Unknown",
                        'precip_mm': hour.precipMM,
                        'humidity': hour.humidity,
                        'visibility': hour.visibility,
                        'wind_speed_kmph': hour.windspeedKmph,
                        'wind_dir': hour.winddir16Point,
                        'chance_of_rain': hour.chanceofrain,
                        'chance_of_snow': hour.chanceofsnow
                    }
                    forecast_day['hourly'].append(hourly_data)

                weather_data['forecast'].append(forecast_day)

        return weather_data

    except Exception as e:
        print(f"❌ Error retrieving weather data: {str(e)}")
        return None


def format_weather_summary(weather_data):
    """
    Format weather data into a readable summary string.

    Args:
        weather_data (dict): Weather data dictionary

    Returns:
        str: Formatted weather summary
    """
    if not weather_data:
        return "No weather data available."

    summary = []

    # Location header
    if weather_data['area_info']:
        area = weather_data['area_info']
        summary.append(f"\n🌍 Weather for {area['area_name']}, {area['country']}")
        summary.append("=" * 50)

    # Current conditions
    if weather_data['current_condition']:
        current = weather_data['current_condition']
        summary.append(f"\n📍 Current Conditions:")
        summary.append(f"   Temperature: {current['temp_C']}°C ({current['temp_F']}°F)")
        summary.append(f"   Feels Like: {current['feels_like_C']}°C ({current['feels_like_F']}°F)")
        summary.append(f"   Conditions: {current['weather_desc']}")
        summary.append(f"   Humidity: {current['humidity']}%")
        summary.append(f"   Wind: {current['wind_speed_kmph']} km/h {current['wind_dir']}")
        summary.append(f"   Precipitation: {current['precip_mm']} mm")
        summary.append(f"   UV Index: {current['uv_index']}")

    # Forecast
    if weather_data['forecast']:
        summary.append(f"\n📅 {len(weather_data['forecast'])}-Day Forecast:")
        for day in weather_data['forecast']:
            summary.append(f"\n   Date: {day['date']}")
            summary.append(f"   Temp: {day['min_temp_C']}°C - {day['max_temp_C']}°C")
            summary.append(f"   Avg: {day['avg_temp_C']}°C")
            if float(day['total_snow_cm']) > 0:
                summary.append(f"   Snow: {day['total_snow_cm']} cm")

    return "\n".join(summary)


print("✅ Weather data functions loaded successfully!")

## 📊 Visualization Functions <a id='visualization'></a>

Functions for creating visual representations of weather data.

In [ ]:
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    if not weather_data or not weather_data['forecast']:
        print("❌ No forecast data available for visualization")
        return None

    # Extract data for plotting
    dates = []
    max_temps = []
    min_temps = []
    avg_temps = []

    for day in weather_data['forecast']:
        dates.append(datetime.strptime(day['date'], '%Y-%m-%d'))
        max_temps.append(float(day['max_temp_C']))
        min_temps.append(float(day['min_temp_C']))
        avg_temps.append(float(day['avg_temp_C']))

    # Create the figure
    fig, ax = plt.subplots(figsize=(12, 6))

    # Plot the temperature data
    ax.plot(dates, max_temps, marker='o', linewidth=2, label='Max Temperature', color='#FF6B6B')
    ax.plot(dates, avg_temps, marker='s', linewidth=2, label='Avg Temperature', color='#4ECDC4')
    ax.plot(dates, min_temps, marker='v', linewidth=2, label='Min Temperature', color='#95E1D3')

    # Fill between max and min for visual appeal
    ax.fill_between(dates, min_temps, max_temps, alpha=0.2, color='#4ECDC4')

    # Formatting
    ax.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax.set_ylabel('Temperature (°C)', fontsize=12, fontweight='bold')

    location_name = weather_data['area_info']['area_name'] if weather_data['area_info'] else weather_data['location']
    ax.set_title(f'Temperature Forecast for {location_name}', fontsize=14, fontweight='bold', pad=20)

    # Format x-axis dates
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    plt.xticks(rotation=45)

    # Add grid for better readability
    ax.grid(True, alpha=0.3, linestyle='--')

    # Add legend
    ax.legend(loc='best', framealpha=0.9)

    # Adjust layout
    plt.tight_layout()

    if output_type == 'figure':
        return fig
    else:
        plt.show()
        return None


def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    if not weather_data or not weather_data['forecast']:
        print("❌ No forecast data available for visualization")
        return None

    # Extract hourly precipitation chances
    dates = []
    rain_chances = []

    for day in weather_data['forecast']:
        date = datetime.strptime(day['date'], '%Y-%m-%d')

        # Get hourly data for each day
        for hour in day['hourly']:
            time_str = hour['time']
            hour_num = int(time_str) // 100
            datetime_obj = date.replace(hour=hour_num)
            dates.append(datetime_obj)
            rain_chances.append(float(hour['chance_of_rain']))

    # Create the figure
    fig, ax = plt.subplots(figsize=(14, 6))

    # Create bar chart
    colors = ['#3498db' if chance < 30 else '#f39c12' if chance < 60 else '#e74c3c'
              for chance in rain_chances]
    ax.bar(dates, rain_chances, width=0.03, color=colors, alpha=0.7, edgecolor='black', linewidth=0.5)

    # Add threshold lines
    ax.axhline(y=30, color='orange', linestyle='--', alpha=0.5, label='Low Risk (30%)')
    ax.axhline(y=60, color='red', linestyle='--', alpha=0.5, label='High Risk (60%)')

    # Formatting
    ax.set_xlabel('Date and Time', fontsize=12, fontweight='bold')
    ax.set_ylabel('Chance of Rain (%)', fontsize=12, fontweight='bold')

    location_name = weather_data['area_info']['area_name'] if weather_data['area_info'] else weather_data['location']
    ax.set_title(f'Precipitation Forecast for {location_name}', fontsize=14, fontweight='bold', pad=20)

    # Format x-axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_minor_locator(mdates.HourLocator(interval=6))
    plt.xticks(rotation=45)

    # Set y-axis limits
    ax.set_ylim(0, 100)

    # Add grid
    ax.grid(True, alpha=0.3, linestyle='--', axis='y')

    # Add legend
    ax.legend(loc='upper left', framealpha=0.9)

    # Adjust layout
    plt.tight_layout()

    if output_type == 'figure':
        return fig
    else:
        plt.show()
        return None


def create_combined_visualisation(weather_data):
    """
    Create a combined visualization showing both temperature and precipitation.

    Args:
        weather_data (dict): The processed weather data
    """
    if not weather_data or not weather_data['forecast']:
        print("❌ No forecast data available for visualization")
        return

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

    # Temperature plot (top)
    dates = []
    max_temps = []
    min_temps = []
    avg_temps = []

    for day in weather_data['forecast']:
        dates.append(datetime.strptime(day['date'], '%Y-%m-%d'))
        max_temps.append(float(day['max_temp_C']))
        min_temps.append(float(day['min_temp_C']))
        avg_temps.append(float(day['avg_temp_C']))

    ax1.plot(dates, max_temps, marker='o', linewidth=2, label='Max Temp', color='#FF6B6B')
    ax1.plot(dates, avg_temps, marker='s', linewidth=2, label='Avg Temp', color='#4ECDC4')
    ax1.plot(dates, min_temps, marker='v', linewidth=2, label='Min Temp', color='#95E1D3')
    ax1.fill_between(dates, min_temps, max_temps, alpha=0.2, color='#4ECDC4')
    ax1.set_ylabel('Temperature (°C)', fontsize=11, fontweight='bold')
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.legend(loc='best')

    location_name = weather_data['area_info']['area_name'] if weather_data['area_info'] else weather_data['location']
    ax1.set_title(f'Weather Forecast Dashboard for {location_name}', fontsize=14, fontweight='bold', pad=20)

    # Precipitation plot (bottom)
    precip_dates = []
    rain_chances = []

    for day in weather_data['forecast']:
        date = datetime.strptime(day['date'], '%Y-%m-%d')
        for hour in day['hourly']:
            time_str = hour['time']
            hour_num = int(time_str) // 100
            datetime_obj = date.replace(hour=hour_num)
            precip_dates.append(datetime_obj)
            rain_chances.append(float(hour['chance_of_rain']))

    colors = ['#3498db' if c < 30 else '#f39c12' if c < 60 else '#e74c3c' for c in rain_chances]
    ax2.bar(precip_dates, rain_chances, width=0.03, color=colors, alpha=0.7)
    ax2.axhline(y=30, color='orange', linestyle='--', alpha=0.5, label='Low Risk')
    ax2.axhline(y=60, color='red', linestyle='--', alpha=0.5, label='High Risk')
    ax2.set_xlabel('Date', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Chance of Rain (%)', fontsize=11, fontweight='bold')
    ax2.set_ylim(0, 100)
    ax2.grid(True, alpha=0.3, linestyle='--', axis='y')
    ax2.legend(loc='upper left')

    # Format x-axis for both plots
    for ax in [ax1, ax2]:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
        ax.xaxis.set_major_locator(mdates.DayLocator())
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

    plt.tight_layout()
    plt.show()


print("✅ Visualization functions loaded successfully!")

## 🤖 Natural Language Processing <a id='nlp'></a>

Functions for parsing weather questions and generating natural language responses.

In [ ]:
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    question_lower = question.lower()

    parsed = {
        'original_question': question,
        'location': None,
        'time_period': 'current',
        'weather_attributes': [],
        'question_type': 'general'
    }

    # Extract location using common patterns
    location_patterns = [
        r'in ([A-Za-z\s]+?)(?:\?|$|,| for| on| this| next| during)',
        r'for ([A-Za-z\s]+?)(?:\?|$|,| on| this| next)',
        r'at ([A-Za-z\s]+?)(?:\?|$|,)',
        r'weather in ([A-Za-z\s]+)',
        r'weather for ([A-Za-z\s]+)'
    ]

    for pattern in location_patterns:
        match = re.search(pattern, question_lower)
        if match:
            parsed['location'] = match.group(1).strip()
            break

    # Determine time period
    time_keywords = {
        'current': ['now', 'current', 'currently', 'today', 'right now'],
        'tomorrow': ['tomorrow', 'tmr'],
        'this week': ['this week', 'week', 'weekly'],
        'weekend': ['weekend', 'saturday', 'sunday'],
        'next week': ['next week'],
        'forecast': ['forecast', 'coming days', 'next few days', 'upcoming']
    }

    for period, keywords in time_keywords.items():
        if any(kw in question_lower for kw in keywords):
            parsed['time_period'] = period
            break

    # Identify weather attributes
    attribute_keywords = {
        'temperature': ['temperature', 'temp', 'hot', 'cold', 'warm', 'cool', 'degrees'],
        'precipitation': ['rain', 'snow', 'precipitation', 'wet', 'showers'],
        'wind': ['wind', 'windy', 'breeze', 'gust'],
        'humidity': ['humidity', 'humid', 'moisture'],
        'conditions': ['weather', 'conditions', 'forecast', 'like', 'sunny', 'cloudy'],
        'umbrella': ['umbrella', 'coat', 'jacket'],
        'outdoor': ['hike', 'hiking', 'picnic', 'outdoor', 'outside', 'activities']
    }

    for attribute, keywords in attribute_keywords.items():
        if any(kw in question_lower for kw in keywords):
            if attribute not in parsed['weather_attributes']:
                parsed['weather_attributes'].append(attribute)

    # Determine question type
    if any(word in question_lower for word in ['should', 'need', 'bring']):
        parsed['question_type'] = 'recommendation'
    elif any(word in question_lower for word in ['what', 'how', 'when']):
        parsed['question_type'] = 'query'
    elif any(word in question_lower for word in ['will', 'is it going to']):
        parsed['question_type'] = 'prediction'

    # Default to general conditions if no specific attributes found
    if not parsed['weather_attributes']:
        parsed['weather_attributes'] = ['conditions']

    return parsed


def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    if not weather_data:
        return "I'm sorry, I couldn't retrieve weather data for that location. Please check the location name and try again."

    location_name = weather_data['area_info']['area_name'] if weather_data['area_info'] else parsed_question['location'] or 'your location'
    time_period = parsed_question['time_period']
    attributes = parsed_question['weather_attributes']
    question_type = parsed_question['question_type']

    response_parts = []

    # Build response based on time period and attributes
    if time_period == 'current' or time_period == 'today':
        current = weather_data['current_condition']
        if current:
            if 'temperature' in attributes:
                response_parts.append(f"The current temperature in {location_name} is {current['temp_C']}°C ({current['temp_F']}°F), feeling like {current['feels_like_C']}°C.")

            if 'precipitation' in attributes or 'umbrella' in attributes:
                if float(current['precip_mm']) > 0:
                    response_parts.append(f"It's currently raining with {current['precip_mm']} mm of precipitation.")
                    if 'umbrella' in attributes:
                        response_parts.append("Yes, you should bring an umbrella! ☔")
                else:
                    response_parts.append("There's no precipitation right now.")
                    if 'umbrella' in attributes:
                        response_parts.append("No umbrella needed at the moment! ☀️")

            if 'wind' in attributes:
                response_parts.append(f"Wind is blowing at {current['wind_speed_kmph']} km/h from the {current['wind_dir']}.")

            if 'humidity' in attributes:
                response_parts.append(f"The humidity level is {current['humidity']}%.")

            if 'conditions' in attributes or not response_parts:
                response_parts.insert(0, f"Currently in {location_name}, it's {current['weather_desc'].lower()} with a temperature of {current['temp_C']}°C.")

    elif time_period == 'tomorrow' and weather_data['forecast']:
        if len(weather_data['forecast']) > 1:
            tomorrow = weather_data['forecast'][1]
        else:
            tomorrow = weather_data['forecast'][0]

        if 'temperature' in attributes:
            response_parts.append(f"Tomorrow in {location_name}, temperatures will range from {tomorrow['min_temp_C']}°C to {tomorrow['max_temp_C']}°C.")

        if 'precipitation' in attributes or 'umbrella' in attributes:
            # Check hourly data for rain chances
            max_rain_chance = max([float(h['chance_of_rain']) for h in tomorrow['hourly']])
            if max_rain_chance > 60:
                response_parts.append(f"There's a high chance of rain tomorrow ({max_rain_chance}%).")
                if 'umbrella' in attributes:
                    response_parts.append("Definitely bring an umbrella! ☔")
            elif max_rain_chance > 30:
                response_parts.append(f"There's a moderate chance of rain tomorrow ({max_rain_chance}%).")
                if 'umbrella' in attributes:
                    response_parts.append("It might be wise to bring an umbrella just in case. 🌂")
            else:
                response_parts.append(f"Low chance of rain tomorrow ({max_rain_chance}%).")
                if 'umbrella' in attributes:
                    response_parts.append("You probably won't need an umbrella. ☀️")

        if 'outdoor' in attributes:
            avg_temp = float(tomorrow['avg_temp_C'])
            max_rain = max([float(h['chance_of_rain']) for h in tomorrow['hourly']])

            if 15 <= avg_temp <= 25 and max_rain < 30:
                response_parts.append("Tomorrow looks great for outdoor activities! The weather should be pleasant. 🌤️")
            elif avg_temp < 5 or avg_temp > 35:
                response_parts.append("Tomorrow might be a bit extreme for outdoor activities. Consider indoor alternatives.")
            elif max_rain > 60:
                response_parts.append("Tomorrow might not be ideal for outdoor activities due to high rain chances. Have a backup plan! 🌧️")
            else:
                response_parts.append("Tomorrow should be okay for outdoor activities, though conditions may not be perfect.")

    elif time_period in ['forecast', 'this week', 'next week'] and weather_data['forecast']:
        num_days = len(weather_data['forecast'])
        temps = [float(day['avg_temp_C']) for day in weather_data['forecast']]
        avg_temp = sum(temps) / len(temps)

        response_parts.append(f"Over the next {num_days} days in {location_name}:")

        if 'temperature' in attributes:
            min_temp = min([float(day['min_temp_C']) for day in weather_data['forecast']])
            max_temp = max([float(day['max_temp_C']) for day in weather_data['forecast']])
            response_parts.append(f"Temperatures will range from {min_temp}°C to {max_temp}°C, averaging around {avg_temp:.1f}°C.")

        if 'precipitation' in attributes:
            rainy_days = 0
            for day in weather_data['forecast']:
                max_rain = max([float(h['chance_of_rain']) for h in day['hourly']])
                if max_rain > 50:
                    rainy_days += 1

            if rainy_days > num_days // 2:
                response_parts.append(f"Expect rain on {rainy_days} of the {num_days} days. Keep that umbrella handy! 🌧️")
            elif rainy_days > 0:
                response_parts.append(f"Rain is likely on {rainy_days} days.")
            else:
                response_parts.append("Mostly dry weather expected! ☀️")

    # If no specific response was generated, provide a general summary
    if not response_parts:
        current = weather_data['current_condition']
        if current:
            response_parts.append(f"In {location_name}, it's currently {current['weather_desc'].lower()} with a temperature of {current['temp_C']}°C ({current['temp_F']}°F).")

    return " ".join(response_parts)


def ask_weather_ai(question, weather_data):
    """
    Use AI to generate a more sophisticated response to weather questions.
    Falls back to rule-based response if AI is unavailable.

    Args:
        question (str): User's question
        weather_data (dict): Weather data context

    Returns:
        str: AI-generated or rule-based response
    """
    try:
        # Check if API key is set
        if not os.environ.get('HANDS_ON_AI_API_KEY'):
            # Fall back to rule-based response
            parsed = parse_weather_question(question)
            return generate_weather_response(parsed, weather_data)

        # Prepare context for AI
        context = format_weather_summary(weather_data)

        prompt = f"""You are a helpful weather assistant. Answer the following question based on the weather data provided.

Weather Data:
{context}

User Question: {question}

Provide a natural, friendly response that directly answers the user's question. Be concise and helpful."""

        response = get_response(prompt)
        return response

    except Exception as e:
        print(f"⚠️ AI response failed, using rule-based response: {str(e)}")
        parsed = parse_weather_question(question)
        return generate_weather_response(parsed, weather_data)


print("✅ Natural language processing functions loaded successfully!")

## 🧭 User Interface <a id='ui'></a>

Interactive menu system for the Weather Advisor application.

In [ ]:
def display_welcome():
    """Display welcome message and application information."""
    print("\n" + "="*60)
    print("🌦️  WEATHERWISE - INTELLIGENT WEATHER ADVISORY SYSTEM  🌦️")
    print("="*60)
    print("\nWelcome to WeatherWise! Your intelligent weather companion.")
    print("\nFeatures:")
    print("  📍 Get weather for any location")
    print("  📊 View temperature and precipitation visualizations")
    print("  💬 Ask questions in natural language")
    print("  📅 Get multi-day forecasts")
    print("\n" + "="*60 + "\n")


def main_menu():
    """
    Display and handle the main menu.

    Returns:
        int: User's menu choice
    """
    print("\n📋 MAIN MENU")
    print("-" * 40)
    menu_options = [
        "View Current Weather",
        "View Weather Forecast",
        "Show Temperature Visualization",
        "Show Precipitation Visualization",
        "Show Combined Dashboard",
        "Ask Weather Question (Natural Language)",
        "Change Location",
        "Exit"
    ]

    for i, option in enumerate(menu_options, 1):
        print(f"  {i}. {option}")

    choice = pyip.inputInt("\nEnter your choice (1-8): ", min=1, max=8)
    return choice


def get_location_input():
    """
    Get location input from user.

    Returns:
        str: Location name
    """
    print("\n📍 Enter Location")
    print("-" * 40)
    print("Examples: London, New York, Tokyo, Paris")
    print("Leave blank for current location (based on IP)\n")

    location = pyip.inputStr("Location: ", blank=True)

    if not location:
        location = ""  # Empty string for current location
        print("Using current location...")

    return location


def get_forecast_days():
    """
    Get number of forecast days from user.

    Returns:
        int: Number of days (1-5)
    """
    print("\nHow many days of forecast would you like?")
    days = pyip.inputInt("Enter number of days (1-5): ", min=1, max=5)
    return days


def display_current_weather(weather_data):
    """
    Display current weather conditions in a formatted way.

    Args:
        weather_data (dict): Weather data dictionary
    """
    if not weather_data or not weather_data['current_condition']:
        print("\n❌ No current weather data available.")
        return

    current = weather_data['current_condition']
    area = weather_data['area_info']

    print("\n" + "="*50)
    print(f"☁️  CURRENT WEATHER")
    print("="*50)

    if area:
        print(f"\n📍 Location: {area['area_name']}, {area['country']}")
        if area['region']:
            print(f"   Region: {area['region']}")

    print(f"\n🌡️  Temperature: {current['temp_C']}°C ({current['temp_F']}°F)")
    print(f"   Feels Like: {current['feels_like_C']}°C ({current['feels_like_F']}°F)")
    print(f"\n☁️  Conditions: {current['weather_desc']}")
    print(f"💧 Humidity: {current['humidity']}%")
    print(f"💨 Wind: {current['wind_speed_kmph']} km/h {current['wind_dir']}")
    print(f"🌧️  Precipitation: {current['precip_mm']} mm")
    print(f"👁️  Visibility: {current['visibility']} km")
    print(f"☀️  UV Index: {current['uv_index']}")
    print("\n" + "="*50)


def display_forecast(weather_data):
    """
    Display weather forecast in a formatted way.

    Args:
        weather_data (dict): Weather data dictionary
    """
    if not weather_data or not weather_data['forecast']:
        print("\n❌ No forecast data available.")
        return

    area = weather_data['area_info']

    print("\n" + "="*50)
    print(f"📅 WEATHER FORECAST")
    print("="*50)

    if area:
        print(f"\n📍 Location: {area['area_name']}, {area['country']}")

    for i, day in enumerate(weather_data['forecast'], 1):
        date_obj = datetime.strptime(day['date'], '%Y-%m-%d')
        day_name = date_obj.strftime('%A')

        print(f"\n{'─'*50}")
        print(f"Day {i}: {day_name}, {day['date']}")
        print(f"{'─'*50}")
        print(f"🌡️  Temperature: {day['min_temp_C']}°C - {day['max_temp_C']}°C (Avg: {day['avg_temp_C']}°C)")
        print(f"☀️  Sun Hours: {day['sun_hour']} hours")
        print(f"☀️  UV Index: {day['uv_index']}")

        if float(day['total_snow_cm']) > 0:
            print(f"❄️  Snow: {day['total_snow_cm']} cm")

        # Show peak rain chance
        max_rain = max([float(h['chance_of_rain']) for h in day['hourly']])
        print(f"🌧️  Max Rain Chance: {max_rain}%")

    print("\n" + "="*50)


print("✅ User interface functions loaded successfully!")

## 🧩 Main Application Logic <a id='main-app'></a>

The main application that ties all components together.

In [ ]:
def run_weather_advisor():
    """
    Main application loop for the Weather Advisor.
    """
    # Display welcome message
    display_welcome()

    # Get initial location
    current_location = get_location_input()

    # Fetch initial weather data
    print("\n🔄 Fetching weather data...")
    weather_data = get_weather_data(current_location, forecast_days=5)

    if not weather_data:
        print("\n❌ Failed to fetch weather data. Please try again.")
        return

    print("\n✅ Weather data loaded successfully!")

    # Main application loop
    while True:
        try:
            choice = main_menu()

            if choice == 1:
                # View Current Weather
                display_current_weather(weather_data)

            elif choice == 2:
                # View Weather Forecast
                display_forecast(weather_data)

            elif choice == 3:
                # Show Temperature Visualization
                print("\n📊 Generating temperature visualization...")
                create_temperature_visualisation(weather_data)

            elif choice == 4:
                # Show Precipitation Visualization
                print("\n📊 Generating precipitation visualization...")
                create_precipitation_visualisation(weather_data)

            elif choice == 5:
                # Show Combined Dashboard
                print("\n📊 Generating combined weather dashboard...")
                create_combined_visualisation(weather_data)

            elif choice == 6:
                # Ask Weather Question
                print("\n💬 Natural Language Weather Query")
                print("-" * 50)
                print("Examples:")
                print("  - Should I bring an umbrella tomorrow?")
                print("  - What's the temperature going to be like this week?")
                print("  - Is it good weather for hiking tomorrow?")
                print("")

                question = pyip.inputStr("Your question: ")

                print("\n🤔 Processing your question...\n")

                # Try AI response first, fall back to rule-based
                response = ask_weather_ai(question, weather_data)

                print("\n💬 Response:")
                print("-" * 50)
                print(response)
                print("-" * 50)

            elif choice == 7:
                # Change Location
                current_location = get_location_input()
                forecast_days = get_forecast_days()

                print("\n🔄 Fetching weather data...")
                weather_data = get_weather_data(current_location, forecast_days)

                if not weather_data:
                    print("\n❌ Failed to fetch weather data. Keeping previous location.")
                else:
                    print("\n✅ Weather data updated successfully!")

            elif choice == 8:
                # Exit
                print("\n👋 Thank you for using WeatherWise! Stay safe and enjoy the weather!")
                print("="*60 + "\n")
                break

            # Pause before showing menu again
            input("\n\nPress Enter to continue...")

        except KeyboardInterrupt:
            print("\n\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ An error occurred: {str(e)}")
            print("Please try again.")


print("✅ Main application logic loaded successfully!")
print("\n🚀 Ready to run! Execute the cell below to start the Weather Advisor.")

## 🧪 Testing and Examples <a id='testing'></a>

Test individual functions and demonstrate key features.

### Test 1: Weather Data Retrieval

In [ ]:
# Test getting weather data for a specific location
print("Testing weather data retrieval for London...\n")
test_weather = get_weather_data("London", forecast_days=3)

if test_weather:
    print("✅ Successfully retrieved weather data!")
    print(format_weather_summary(test_weather))
else:
    print("❌ Failed to retrieve weather data")

### Test 2: Temperature Visualization

In [ ]:
# Test temperature visualization
if test_weather:
    print("Testing temperature visualization...\n")
    create_temperature_visualisation(test_weather)

### Test 3: Precipitation Visualization

In [ ]:
# Test precipitation visualization
if test_weather:
    print("Testing precipitation visualization...\n")
    create_precipitation_visualisation(test_weather)

### Test 4: Natural Language Processing

In [ ]:
# Test parsing weather questions
test_questions = [
    "Should I bring an umbrella tomorrow in London?",
    "What's the temperature going to be like this week?",
    "Will it rain tomorrow?",
    "Is it good weather for hiking this weekend?"
]

print("Testing natural language question parsing...\n")

for question in test_questions:
    print(f"Question: {question}")
    parsed = parse_weather_question(question)
    print(f"Parsed: {parsed}")
    print()

    if test_weather:
        response = generate_weather_response(parsed, test_weather)
        print(f"Response: {response}")
        print("-" * 60)
        print()

### Test 5: Combined Dashboard

In [ ]:
# Test combined dashboard
if test_weather:
    print("Testing combined weather dashboard...\n")
    create_combined_visualisation(test_weather)

### Test 6: Error Handling

In [ ]:
# Test error handling with invalid location
print("Testing error handling with invalid location...\n")
invalid_weather = get_weather_data("ThisLocationDefinitelyDoesNotExist12345", forecast_days=3)

if invalid_weather:
    print("Received data (might be fallback/mock data)")
else:
    print("✅ Error handling worked correctly - no data returned for invalid location")

## 🚀 Run the Application

Execute the cell below to start the interactive Weather Advisor application!

In [ ]:
# Run the main application
run_weather_advisor()

In [ ]:
# Test: My personal cell with wind + sunrise/sunset (Perth/Lisbon)

import requests
from datetime import datetime

cities = ["Perth", "Lisbon"]
url = "https://api.open-meteo.com/v1/forecast"

for city in cities:
    print(f"\n--- {city.upper()} ---")
    params = {
        "latitude": -31.95 if city == "Perth" else 38.72,
        "longitude": 115.86 if city == "Perth" else -9.14,
        "current_weather": True,
        "timezone": "auto",
        "daily": ["sunrise", "sunset"]
    }
    response = requests.get(url, params=params)
    data = response.json()
    weather = data.get("current_weather", {})
    daily = data.get("daily", {})

    print(f"Temperature: {weather.get('temperature', 'N/A')}°C")
    print(f"Wind: {weather.get('windspeed', 'N/A')} km/h")
    print(f"Wind direction: {weather.get('winddirection', 'N/A')}°")
    print(f"Sunrise: {daily.get('sunrise', ['N/A'])[0]}")
    print(f"Sunset: {daily.get('sunset', ['N/A'])[0]}")


## 📝 Project Reflection

This Weather Advisor application demonstrates:

1. **Modular Design**: Functions are organized into logical categories (data retrieval, visualization, NLP, UI)
2. **Data Visualization**: Multiple chart types show temperature trends and precipitation patterns
3. **Natural Language Processing**: Parses user questions and generates contextual responses
4. **User Experience**: Intuitive menu system with clear options and error handling
5. **API Integration**: Seamless integration with fetch-my-weather package
6. **Error Handling**: Graceful handling of network issues and invalid inputs

### Key Features:
- ✅ Real-time weather data retrieval
- ✅ 5-day weather forecast
- ✅ Temperature and precipitation visualizations
- ✅ Natural language question answering
- ✅ Interactive menu-based interface
- ✅ Comprehensive error handling
- ✅ Clean, well-documented code

### Technologies Used:
- Python 3.x
- fetch-my-weather (weather data API)
- matplotlib (data visualization)
- pyinputplus (user input validation)
- hands-on-ai (optional AI enhancements)

This project successfully combines technical Python skills with practical application development!